# **PART 1 Web Scrapping urls**



Get url for firms on a specific date (given) **2017**

RETURN a dataframe containing:


1.   cik number
2.   company name
3.   form id (should be 10-K)
4.   date
5.   file_url



In [ ]:
# All filings for particular year and quarter, pulling necessary filings
#import libraries
import requests
import urllib
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
import os
import pandas as pd
path = "/content/drive/Shared drives/AFP/Web Scrapping/2017"
os.chdir(path)

fullContent_10k_2017 = pd.read_csv("FullContent_10k_2017.csv")

In [ ]:
fullContent_10k_2017 = fullContent_10k_2017.iloc[:, 1:8]

In [ ]:

#lets first make a function that will make the process of building url easy

def make_url(base_url, comp): # basic self-define FUN when webscrapping
    
    url = base_url
    
    #add each component to base url
    
    for r in comp:
        url = '{}/{}'.format(url,r) # equivalent to '%s/%s' % ('one', 'two')
        
    return url



In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/Shared drives/AFP/Web Scrapping/2017"

os.chdir(path)

In [ ]:
#base url for daily index files
base_url = r"https://www.sec.gov/Archives/edgar/daily-index"

#### 2017
#create the daily index url for 2019
year_url = make_url(base_url , ['2017', 'index.json']) # make_url: self-define FUN
year_url

In [ ]:
#request the 2019 url
content = requests.get(year_url)
decoded_content = content.json()
decoded_content

#data frame to store the links
url_2017=[]
# get all urls for 2019

#loop through dictionary
for item in decoded_content['directory']['item']:
    
    #get name of the folder
    print("-"*100)
    print('Pulling url for the quarter {}'.format(item['name']))
    
    #create the qrt url
    qtr_url = make_url(base_url , ['2017',item['name'], 'index.json'])
    
    print(qtr_url)
    
        #request url and decode it
    file_content = requests.get(qtr_url)
    decoded_content = file_content.json()
    
#   print("Pulling files")

    ## decode qtr_url as decoded_content 
    for file in decoded_content['directory']['item']:
        #print("File:", file)
        file_url = make_url(base_url , ['2017',item['name'], file['name']])
        if file_url[-19:-13] == "master" :
            print(file_url)
            url_2017.append([file_url,file_url[-12:-4]])
            #print(file_url[-19:-13])
url_2017=pd.DataFrame(url_2017)   

In [ ]:
#define a master file url
master_data = []

In [ ]:

for url in url_2017[0][141:142]:
    file_url = url

    #make a request for that file
    content = requests.get(file_url).content
    # print(content)

    #Lets write the content to text file
    with open('master.txt','wb') as f: ## wirte in bonary mode, so reading also require binary mode
        f.write(content)
    #downlad multiple text files

    #lets read the content to a text file
    with open('master.txt','rb') as f:
        byte_data = f.read()
    # print(byte_data)

    #decode the byte data
    data=byte_data.decode('utf-8').split('  ')
    #print(data)
    #we now have list

    #finding the starting index
    for index, item in enumerate(data):

        if 'ftp://ftp.sec.gov/edgar/' in item:
            start_ind = index

    #create a new list that removes the junk

    # data_format = data[start_ind+1:]

    #----------------------------
    data_format = data[start_ind:]
    #print(len(data_format))
    #----------------------------


    #loop through the data list
    for index, item in enumerate(data_format):

            if index == 0:
                clean_item_data = item.replace('\n','|').split('|')
                clean_item_data = clean_item_data[8:] ## exculde header
            else:
                clean_item_data = item.replace('\n','|').split('|')

            for index , row in enumerate(clean_item_data):

                #when you find txt.file

                if '.txt' in row:

                    mini_list = clean_item_data[(index - 4): index+1] ## for each entree, from CIK to FILE 

                    if len(mini_list) != 0:
                        mini_list[4] = "http://www.sec.gov/Archives/" + mini_list[4]
                        master_data.append(mini_list)
                       

In [ ]:
#loop through master data set

for index,document in enumerate(master_data):
    
    #create dictionary
    document_dict = {}
    document_dict["cik number"] = document[0]
    document_dict["company name"] = document[1]
    document_dict["form id"] = document[2]
    document_dict["date"] = document[3]
    document_dict["file_url"] = document[4]
    
    master_data[index] = document_dict
    

In [ ]:
for document_dict in master_data:
    
    if document_dict['form id'] == '10-K':
        print(document_dict['company name'])
        print(document_dict['file_url'])

In [ ]:
document_df = pd.DataFrame.from_dict(master_data)
document_df.head(5)

document_df_10k = document_df[document_df['form id'] == '10-K']
document_df_10k = document_df_10k.reset_index(drop=True)
print(document_df_10k.shape[0])
print(document_df_10k.head(5))

document_df_10k.to_csv("master_10k_2017.csv")


# **PART 2 Parsing SEC Filings**



1. Get the full 10-K text for firms on a specific date (given)

    RETURN a dataframe with one more column (10KText_All)

    original text (no lowercase check)
    1.   cik number
    2.   company name
    3.   form id (should be 10-K)
    4.   date
    5.   file_url
    6.   10KText_All

In [ ]:
def Get_10k_FullContent(fileUrl):
          
    # define the url to specific html_text file
    new_html_text = fileUrl
    # print(new_html_text)

    # grab the response
    response = requests.get(new_html_text)
    # print(response)

    # pass it through the parser, in this case let's just use lxml because the tags seem to follow xml.
    soup = BeautifulSoup(response.content, 'lxml')

    # get the first document tag, since 10-K file is always within the first one.
    soup_10k = soup.find('document')

    # parse the text within the document tag
    document_fulltext_10k = soup_10k.get_text()
    # simple data cleaning
    document_fulltext_10k = document_fulltext_10k.replace("\n", " ").replace("\xa0", " ").replace("\t", " ").replace('☐', ' ').replace('☒', ' ') 

    return document_fulltext_10k

In [ ]:
document_df_10k = pd.read_csv("master_10k_2017.csv")

In [ ]:
document_df_10k['10KText_All'] = document_df_10k['file_url'].apply(Get_10k_FullContent)

In [ ]:
document_df_10k.to_csv("FullContent_10k_2017.csv")